In [ ]:
import os
import sys
import pandas as pd
import numpy as np
import polars as pl
import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder
import pickle
import warnings
warnings.filterwarnings('ignore')

# パスの設定
print(f"現在のディレクトリ: {os.getcwd()}")
sys.path.append(os.getcwd())
sys.path.append('../../data')

print("ライブラリのインポート完了")


In [ ]:
# グローバル変数の初期化
global_models = None
global_le = None
global_feature_cols = None

def load_saved_model():
    """
    保存されたモデルを読み込み
    """
    global global_models, global_le, global_feature_cols
    
    # モデルファイルのパスを設定
    model_dir = '../../output/experiments/baseline_lightgbm_v1/models/'
    
    print(f"モデルディレクトリ: {model_dir}")
    
    try:
        # 保存されたモデルを読み込み
        with open(os.path.join(model_dir, 'trained_models.pkl'), 'rb') as f:
            global_models = pickle.load(f)
        
        # ラベルエンコーダーを読み込み
        with open(os.path.join(model_dir, 'label_encoder.pkl'), 'rb') as f:
            global_le = pickle.load(f)
        
        # 特徴量カラムを読み込み
        with open(os.path.join(model_dir, 'feature_cols.pkl'), 'rb') as f:
            global_feature_cols = pickle.load(f)
        
        print("保存されたモデルの読み込み完了")
        print(f"モデル数: {len(global_models)}")
        print(f"クラス数: {len(global_le.classes_)}")
        print(f"特徴量数: {len(global_feature_cols)}")
        
        # クラス名を表示
        print("\n予測可能なジェスチャー:")
        for i, class_name in enumerate(global_le.classes_):
            print(f"  {i:2d}: {class_name}")
        
    except FileNotFoundError as e:
        print(f"モデルファイルが見つかりません: {e}")
        print("事前にモデルを学習・保存してください")
        raise e

# モデルを読み込み
load_saved_model()


In [ ]:
def extract_features(df):
    """
    シーケンス単位の特徴量抽出
    """
    # センサーデータのカラムを特定
    sensor_cols = [col for col in df.columns if any(sensor in col for sensor in ['acc_', 'rot_', 'tof_', 'thm_'])]
    
    # シーケンス単位で統計量を計算
    features = {}
    
    for col in sensor_cols:
        if df[col].dtype in ['float64', 'int64']:
            # 基本統計量
            features[f'{col}_mean'] = df[col].mean()
            features[f'{col}_std'] = df[col].std()
            features[f'{col}_min'] = df[col].min()
            features[f'{col}_max'] = df[col].max()
            features[f'{col}_median'] = df[col].median()
            
            # 分位数
            features[f'{col}_q25'] = df[col].quantile(0.25)
            features[f'{col}_q75'] = df[col].quantile(0.75)
            
            # 範囲
            features[f'{col}_range'] = df[col].max() - df[col].min()
            
            # 歪度と尖度
            features[f'{col}_skew'] = df[col].skew()
            features[f'{col}_kurtosis'] = df[col].kurtosis()
    
    return features

print("特徴量抽出関数の定義完了")


In [ ]:
def predict_gesture(sequence_df):
    """
    シーケンスデータからジェスチャーを予測
    """
    global global_models, global_le, global_feature_cols
    
    # モデルが正しく読み込まれているかチェック
    if global_models is None or global_le is None or global_feature_cols is None:
        raise ValueError("モデルの読み込みに失敗しました")
    
    # 特徴量抽出
    features = extract_features(sequence_df)
    
    # 特徴量をDataFrameに変換
    feature_df = pd.DataFrame([features])
    
    # 必要な特徴量カラムのみ選択
    feature_df = feature_df[global_feature_cols]
    
    # 予測（アンサンブル）
    predictions = []
    for model in global_models:
        pred = model.predict(feature_df, num_iteration=model.best_iteration)
        predictions.append(pred)
    
    # 平均予測
    avg_pred = np.mean(predictions, axis=0)
    predicted_class = np.argmax(avg_pred)
    
    # クラス名に変換
    predicted_gesture = global_le.inverse_transform([predicted_class])[0]
    
    return predicted_gesture, avg_pred

print("推論関数の定義完了")


In [ ]:
# テストデータの読み込み
test_data = pd.read_csv('../../data/test.csv')
test_demographics = pd.read_csv('../../data/test_demographics.csv')

print(f"テストデータの形状: {test_data.shape}")
print(f"テスト人口統計データの形状: {test_demographics.shape}")

# シーケンスIDを確認
test_sequence_ids = test_data['sequence_id'].unique()
print(f"テストシーケンス数: {len(test_sequence_ids)}")
print(f"テストシーケンスID: {test_sequence_ids}")

# データの基本情報を表示
print("\nテストデータの基本情報:")
print(test_data.head())


In [ ]:
# 各テストシーケンスで推論をテスト
test_results = []

for sequence_id in test_sequence_ids:
    print(f"\n=== シーケンス {sequence_id} の推論テスト ===")
    
    # 該当シーケンスのデータを取得
    sequence_data = test_data[test_data['sequence_id'] == sequence_id]
    
    print(f"シーケンスデータ数: {len(sequence_data)}")
    
    # 推論実行
    try:
        predicted_gesture, prediction_probs = predict_gesture(sequence_data)
        
        print(f"予測結果: {predicted_gesture}")
        print(f"予測確率: {prediction_probs.max():.4f}")
        
        # 結果を保存
        test_results.append({
            'sequence_id': sequence_id,
            'predicted_gesture': predicted_gesture,
            'confidence': prediction_probs.max(),
            'prediction_probs': prediction_probs
        })
        
        # 上位3の予測を表示
        top_3_indices = np.argsort(prediction_probs)[-3:][::-1]
        print("予測トップ3:")
        for i, idx in enumerate(top_3_indices):
            gesture_name = global_le.inverse_transform([idx])[0]
            prob = prediction_probs[idx]
            print(f"  {i+1}: {gesture_name} ({prob:.4f})")
        
    except Exception as e:
        print(f"エラーが発生しました: {e}")
        test_results.append({
            'sequence_id': sequence_id,
            'predicted_gesture': 'ERROR',
            'confidence': 0.0,
            'error': str(e)
        })

print("\n=== 全シーケンスの推論テスト完了 ===")


In [ ]:
# 結果をDataFrameに変換
results_df = pd.DataFrame(test_results)
print("推論結果の概要:")
print(results_df[['sequence_id', 'predicted_gesture', 'confidence']])

# 提出ファイル形式の作成
submission_df = results_df[['sequence_id', 'predicted_gesture']].copy()
submission_df.columns = ['sequence_id', 'gesture']

print("\n提出ファイル形式:")
print(submission_df)

# 提出ファイルを保存
submission_df.to_csv('local_test_submission.csv', index=False)
print("\n提出ファイルを 'local_test_submission.csv' に保存しました")


In [ ]:
print("=== ローカルテスト完了 ===")
print(f"✅ 学習済みモデルの読み込み: 成功 ({len(global_models)}個のモデル)")
print(f"✅ 特徴量抽出: 成功 ({len(global_feature_cols)}個の特徴量)")
print(f"✅ 推論テスト: 成功 ({len(test_results)}個のシーケンス)")
print(f"✅ 提出ファイル作成: 成功 (local_test_submission.csv)")

# エラーがあった場合の報告
error_count = sum(1 for result in test_results if result['predicted_gesture'] == 'ERROR')
if error_count > 0:
    print(f"⚠️  エラーが発生したシーケンス: {error_count}個")
else:
    print("✅ 全シーケンスで正常に推論完了")

print("\n次のステップ:")
print("1. 結果を確認し、推論が正常に動作することを確認")
print("2. cmi-2025-baseline-submission-final.ipynb を Kaggle に提出")
print("3. Kaggle環境でのテストを実行")

print("\n🎉 ローカルテストが完了しました！")
